In [0]:
# https://keras.io/
# !pip install -q keras
#import keras

In [1]:
# import tensorflow as tf
# import tensorflow.keras as keras
# from tensorflow.keras.datasets import cifar10
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, Activation,GlobalAveragePooling2D
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from tensorflow.keras.layers import Concatenate
# from tensorflow.keras.optimizers import Adam,SGD
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import os
# import numpy as np
# from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint,ReduceLROnPlateau


import keras as keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, Activation,GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam,SGD
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from keras.callbacks import LearningRateScheduler,ModelCheckpoint,ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 250
l = 16
num_filter = 32
growth_rate = 12
compression = 0.5
dropout_rate = 0.2
weight_decay = 1e-4
dilate_rate = 1


In [3]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train /= 255.
x_test /= 255.

# for i in range(3):
# 		mean = np.mean(x_train[:, :, :, i])
# 		std = np.std(x_train[:, :, :, i])
# 		x_train[:, :, :, i] = (x_train[:, :, :, i] - mean) / std
# 		x_test[:, :, :, i] = (x_test[:, :, :, i] - mean) / std

170500096/170498071 [==============================] - 83s 0us/step


In [4]:
# x_train_8 = tf.image.resize_images(x_train, [8, 8])
# x_test_8 = tf.image.resize_images(x_test, [8, 8])

# x_train_16 = tf.image.resize_images(x_train, [16, 16])
# x_test_16 = tf.image.resize_images(x_test, [16, 16])

# x_train_24 = tf.image.resize_images(x_train, [24, 24])
# x_test_24 = tf.image.resize_images(x_test, [24, 24])

from  skimage import transform
# new_shape = (8,8,3)
# x_train_8 = np.asarray([transform.resize(image, new_shape) for image in x_train])
# x_test_8 = np.asarray([transform.resize(image, new_shape) for image in x_test])

# new_shape = (16,16,3)
# x_train_16 = np.asarray([transform.resize(image, new_shape) for image in x_train])
# x_test_16 = np.asarray([transform.resize(image, new_shape) for image in x_test])

new_shape = (24,24,3)
x_train_24 = np.asarray([transform.resize(image, new_shape) for image in x_train])
x_test_24 = np.asarray([transform.resize(image, new_shape) for image in x_test])

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression, weight_decay, growth_rate

    temp = input
    
    for _ in range(l):
      
        BatchNorm = BatchNormalization(gamma_regularizer=l2(weight_decay),beta_regularizer=l2(weight_decay))(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_1_1 = Conv2D(int(growth_rate*4), (1,1), use_bias=False ,padding='same',
                              kernel_initializer='he_uniform',kernel_regularizer=l2(weight_decay))(relu)
        
        BatchNorm_1_1 = BatchNormalization(gamma_regularizer=l2(weight_decay),beta_regularizer=l2(weight_decay))(Conv2D_1_1)
        relu_1_1 = Activation('relu')(BatchNorm_1_1)
        Conv2D_3_3 = Conv2D(int(growth_rate), (3,3), use_bias=False ,padding='same', dilation_rate = dilate_rate,
                              kernel_initializer='he_uniform',kernel_regularizer=l2(weight_decay))(relu_1_1)
        
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(rate=dropout_rate)(Conv2D_3_3)
        
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        num_filter += growth_rate
        
    return temp , num_filter

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression, weight_decay
    BatchNorm = BatchNormalization(gamma_regularizer=l2(weight_decay),beta_regularizer=l2(weight_decay))(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same',
                              kernel_initializer='he_uniform',kernel_regularizer=l2(weight_decay))(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(rate=dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2),strides=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression, weight_decay
    BatchNorm = BatchNormalization(gamma_regularizer=l2(weight_decay),beta_regularizer=l2(weight_decay))(input)
    relu = Activation('relu')(BatchNorm)
    #AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    #flat = Flatten()(AvgPooling)
    #output = Dense(num_classes, activation='softmax')(flat)
    GloAvgPooling = GlobalAveragePooling2D()(relu)
    output = Dense(num_classes, activation='softmax',
		kernel_regularizer=l2(weight_decay),bias_regularizer=l2(weight_decay))(GloAvgPooling)
    
    return output

In [0]:

#input = Input(shape=(img_height, img_width, channel))
input = Input(shape=(None, None, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same', dilation_rate = dilate_rate,
			kernel_initializer='he_uniform',kernel_regularizer=l2(weight_decay))(input)

First_Block, num_filters = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filters, dropout_rate)

Second_Block,num_filters = add_denseblock(First_Transition, num_filters, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filters, dropout_rate)

# Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
# Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block,num_filters = add_denseblock(Second_Transition,  num_filters, dropout_rate)
output = output_layer(Last_Block)


In [9]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
datagen = ImageDataGenerator(
			featurewise_center=False,  # set input mean to 0 over the dataset
			samplewise_center=False,  # set each sample mean to 0
			featurewise_std_normalization=False,  # divide inputs by std of the dataset
			samplewise_std_normalization=False,  # divide each input by its std
			zca_whitening=False,  # apply ZCA whitening
			rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
			width_shift_range=0.16,  # randomly shift images horizontally (fraction of total width)
			height_shift_range=0.16,  # randomly shift images vertically (fraction of total height)
			horizontal_flip=True,  # randomly flip images
			vertical_flip=False) # randomly flip images
       

In [0]:
# def step_decay(epoch):
# 	initial_lrate = 0.1
# 	lrate = 0.1
# 	if epoch >= 125 and epoch < 187:
# 		lrate = initial_lrate / 10
# 	if epoch >= 187 and epoch < 225:
# 		lrate = initial_lrate / 100
# 	if epoch >= 225:
# 		lrate = initial_lrate / 1000 

def step_decay(epoch):
	initial_lrate = 0.1
	lrate = 0.1
	if epoch >= 50 and epoch < 75:
		lrate = initial_lrate / 10
	if epoch >= 75:
		lrate = initial_lrate / 100
	
	
	return float(lrate)
lrschedular = LearningRateScheduler(step_decay)

In [0]:
# determine Loss function and Optimizer
# model.compile(loss='categorical_crossentropy',
#               optimizer=Adam(),
#               metrics=['accuracy'])
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.1,momentum=0.9,nesterov=True),
              metrics=['accuracy'])

In [0]:
# tpu_model = tf.contrib.tpu.keras_to_tpu_model(model,
#                                               strategy = tf.contrib.tpu.TPUDistributionStrategy(
#                                                   tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://'+os.environ['COLAB_TPU_ADDR'])
#                                               )
#                                              )

In [13]:
# tpu_model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=50,
#                     verbose=1,
#                     validation_data=(x_test, y_test))

model.fit_generator(datagen.flow(x_train_24, y_train,
									 batch_size=64),
						steps_per_epoch=782,epochs=100,verbose=1,validation_data=(x_test_24, y_test),callbacks=[lrschedular])

Epoch 1/100
782/782 [==============================] - 326s 416ms/step - loss: 3.2429 - acc: 0.4444 - val_loss: 3.4902 - val_acc: 0.4631
Epoch 2/100
782/782 [==============================] - 291s 371ms/step - loss: 2.4423 - acc: 0.5988 - val_loss: 2.4560 - val_acc: 0.5905
Epoch 3/100
782/782 [==============================] - 290s 371ms/step - loss: 1.9907 - acc: 0.6604 - val_loss: 2.4168 - val_acc: 0.5705
Epoch 4/100
782/782 [==============================] - 290s 371ms/step - loss: 1.6801 - acc: 0.6988 - val_loss: 1.6037 - val_acc: 0.7131
Epoch 5/100
782/782 [==============================] - 290s 371ms/step - loss: 1.4717 - acc: 0.7219 - val_loss: 1.8094 - val_acc: 0.6365
Epoch 6/100
782/782 [==============================] - 290s 371ms/step - loss: 1.3130 - acc: 0.7411 - val_loss: 1.5699 - val_acc: 0.6593
Epoch 7/100
782/782 [==============================] - 290s 371ms/step - loss: 1.2128 - acc: 0.7528 - val_loss: 1.8968 - val_acc: 0.5861
Epoch 8/100
782/782 [====================

In [14]:
score = model.evaluate(x_test_24, y_test, verbose=1)
print(score)

10000/10000 [==============================] - 27s 3ms/step
[0.48807270069122316, 0.9144]


In [0]:
model.save_weights("DNST_weights_24x24.h5")
model.save("DNST_model_24x24.hdf5")


In [16]:
def step_decay(epoch):
	initial_lrate = 0.01
	lrate = 0.01
	if epoch >= 10:
		lrate = initial_lrate / 10
		
	return float(lrate)
lrschedular = LearningRateScheduler(step_decay)
model.fit_generator(datagen.flow(x_train, y_train,
									 batch_size=64),
						steps_per_epoch=782,epochs=20,verbose=1,callbacks=[lrschedular])

Epoch 1/20
782/782 [==============================] - 383s 489ms/step - loss: 0.3972 - acc: 0.9330
Epoch 2/20
782/782 [==============================] - 381s 487ms/step - loss: 0.3872 - acc: 0.9373
Epoch 3/20
782/782 [==============================] - 381s 487ms/step - loss: 0.3841 - acc: 0.9362
Epoch 4/20
782/782 [==============================] - 381s 488ms/step - loss: 0.3743 - acc: 0.9393
Epoch 5/20
782/782 [==============================] - 381s 488ms/step - loss: 0.3739 - acc: 0.9385
Epoch 6/20
782/782 [==============================] - 381s 487ms/step - loss: 0.3657 - acc: 0.9409
Epoch 7/20
782/782 [==============================] - 381s 487ms/step - loss: 0.3676 - acc: 0.9391
Epoch 8/20
782/782 [==============================] - 382s 489ms/step - loss: 0.3602 - acc: 0.9428
Epoch 9/20
782/782 [==============================] - 382s 489ms/step - loss: 0.3614 - acc: 0.9407
Epoch 10/20
782/782 [==============================] - 383s 489ms/step - loss: 0.3605 - acc: 0.9401
Epoch 11/

In [20]:
def step_decay(epoch):
	
	lrate = 0.001
		
	return float(lrate)
lrschedular = LearningRateScheduler(step_decay)
model.fit_generator(datagen.flow(x_train, y_train,
									 batch_size=64),
						steps_per_epoch=782,epochs=10,verbose=1,callbacks=[lrschedular])

Epoch 1/10
782/782 [==============================] - 384s 491ms/step - loss: 0.2717 - acc: 0.9713
Epoch 2/10
782/782 [==============================] - 383s 490ms/step - loss: 0.2712 - acc: 0.9716
Epoch 3/10
782/782 [==============================] - 383s 490ms/step - loss: 0.2677 - acc: 0.9726
Epoch 4/10
782/782 [==============================] - 383s 489ms/step - loss: 0.2664 - acc: 0.9728
Epoch 5/10
782/782 [==============================] - 384s 491ms/step - loss: 0.2662 - acc: 0.9720
Epoch 6/10
782/782 [==============================] - 384s 491ms/step - loss: 0.2648 - acc: 0.9737
Epoch 7/10
782/782 [==============================] - 384s 492ms/step - loss: 0.2626 - acc: 0.9742
Epoch 8/10
782/782 [==============================] - 386s 493ms/step - loss: 0.2616 - acc: 0.9743
Epoch 9/10
782/782 [==============================] - 387s 494ms/step - loss: 0.2584 - acc: 0.9753
Epoch 10/10
782/782 [==============================] - 388s 497ms/step - loss: 0.2582 - acc: 0.9746


In [22]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print(score)
print('Test loss: %.3f ' % (score[0]))
print('Test accuracy: %.3f ' % (score[1]*100))

10000/10000 [==============================] - 38s 4ms/step
[0.39195930268764495, 0.935]
Test loss: 0.392 
Test accuracy: 93.500 


In [24]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_weights_32x32.h5")
model.save("DNST_weights_32x32.hdf5")
print("Saved model to disk")

Saved model to disk


# References
1. Original Densenet Paper: https://arxiv.org/pdf/1608.06993
2. https://medium.com/intuitionmachine/notes-on-the-implementation-densenet-in-tensorflow-beeda9dd1504
3. https://forums.fast.ai/t/training-a-model-from-scratch-cifar-10/7897
4. https://towardsdatascience.com/densenet-2810936aeebb
5. https://towardsdatascience.com/normalized-direction-preserving-adam-switching-from-adam-to-sgd-and-nesterov-momentum-adam-with-460be5ddf686


# Densenet implementations
1. Original Densenet Implementation: https://github.com/liuzhuang13/DenseNet
2. Fast.ai: http://files.fast.ai/part2/lesson13/densenet-keras.ipynb
3. Github Users
	a. https://github.com/titu1994/DenseNet
	b. https://github.com/flyyufelix/DenseNet-Keras 

# LR Callbacks
1. Cyclic Learning Rate: https://github.com/bckenstler/CLR
2. SGDR: https://gist.github.com/t2kasa/490610116ddb0f3b664458d0e086e643
3. SWATS: https://arxiv.org/pdf/1712.07628 (Implementation not found)
	https://github.com/kweonwooj/papers/issues/76
	https://www.groundai.com/project/improving-generalization-performance-by-switching-from-adam-to-sgd/
	